In [1]:
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
XMIN = 1470
XMAX = 2840
YMIN = 50
YMAX = 2000

In [3]:
def find_centroids(th, min_size):
    if min_size < 1:
        raise ValueError('min_size must be at least 1')
    
    _, cont, _ = cv2.findContours(th, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    M = map(cv2.moments, cont)
    
    cxs = []
    cys = []

    for i, m in enumerate(M):
        if m['m00'] < min_size:
            continue

        cx = int(m['m10']/m['m00'])
        cy = int(m['m01']/m['m00'])

        cxs.append(cx)
        cys.append(cy)
      
    return cxs, cys

In [4]:
def get_masked_window(grayed, cx, cy, size):
    windowed = grayed[cy-size//2:cy+size//2, cx-size//2:cx+size//2]
    _, bw = cv2.threshold(windowed, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    _, cont, _ = cv2.findContours(bw, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    if 0 < len(cont) < 3:
        return windowed * (bw == 0)
    else:
        return None

## TF model

In [6]:
video = cv2.VideoCapture('../data/videos/TandemRun.mp4')

In [8]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# CNN
SIZE = 28

tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=[None, SIZE*SIZE])
y_ = tf.placeholder(tf.float32, [None, 3])

with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, SIZE, SIZE, 1])
    
with tf.name_scope('conv'):
    W_conv = weight_variable([3, 3, 1, 8])
    b_conv = bias_variable([8])
    h_conv = tf.nn.relu(tf.nn.conv2d(x_image, W_conv, strides=[1, 1, 1, 1], padding='SAME'))
    
with tf.name_scope('pool'):
    h_pool = tf.nn.max_pool(h_conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
with tf.name_scope('fc1'):
    W_fc1 = weight_variable([SIZE//2*SIZE//2*8, 100])
    b_fc1 = bias_variable([100])
        
    h_pool_flat = tf.reshape(h_pool, [-1, SIZE//2*SIZE//2*8])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat, W_fc1) + b_fc1)
    
with tf.name_scope('fc2'):
    W_fc2 = weight_variable([100, 3])
    b_fc2 = bias_variable([3])

    y = tf.matmul(h_fc1, W_fc2) + b_fc2
    
saver = tf.train.Saver()

sess = tf.InteractiveSession()
saver.restore(sess, '../data/tf_save/trained_model.ckpt')

INFO:tensorflow:Restoring parameters from ../data/tf_save/trained_model.ckpt


In [14]:
MIN_CONTOUR_SIZE = 45
WINDOW_SIZE = SIZE
frame_count = 0

out_video = cv2.VideoWriter('../data/videos/out.mp4', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (3840, 2160))

start_time = time.time()

while video.isOpened():
    ret, frame = video.read()

    frame_count += 1
    if frame_count % 30 == 0:
        print('Processed {}s of video. Time Elapsed {}s'.format(frame_count//30, time.time()-start_time))
        start_time = time.time()
        if frame_count // 30 > 10:
            break
      
    cropped = frame[YMIN:YMAX, XMIN:XMAX]
    grayed = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
    th = cv2.adaptiveThreshold(grayed, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)
    cxs, cys = find_centroids(th, MIN_CONTOUR_SIZE)
    
    masks = []
    positions = []
    
    for cx, cy in zip(cxs, cys):
        masked = get_masked_window(grayed, cx, cy, WINDOW_SIZE)
        if masked is not None and masked.size == WINDOW_SIZE * WINDOW_SIZE:
            masks.append(masked)
            positions.append((cx, cy))
    # Feed masked windows to trained model for prediction.
    predictions = sess.run(tf.argmax(y, 1), feed_dict={x: [mask.flatten() for mask in masks]})
    for prediction, (cx, cy) in zip(predictions, positions):
        if prediction == 1:
            cv2.rectangle(frame, (cx+XMIN-15, cy+YMIN-15), (cx+XMIN+15, cy+YMIN+15), (0, 255, 0), 3)
       
    out_video.write(frame)

out_video.release()

Processed 1s of video. Time Elapsed 7.989868879318237s
Processed 2s of video. Time Elapsed 7.931120157241821s
Processed 3s of video. Time Elapsed 7.93904709815979s
Processed 4s of video. Time Elapsed 7.921381950378418s
Processed 5s of video. Time Elapsed 7.944221019744873s
Processed 6s of video. Time Elapsed 7.937960386276245s
Processed 7s of video. Time Elapsed 8.165766716003418s
Processed 8s of video. Time Elapsed 8.1509370803833s
Processed 9s of video. Time Elapsed 8.104694843292236s
Processed 10s of video. Time Elapsed 8.066680908203125s
Processed 11s of video. Time Elapsed 8.124304056167603s


In [13]:
sess.close()

In [22]:
out_video.release()